## [Mestrado] 2.2 Análises estatísticas usando instâncias reais (sklearn e autoencoder) com tsfresh

## Bibliotecas e configurações

In [1]:
# Artifício para alcular tempo total do notebook Jupyter
from datetime import datetime 
start_time = datetime.now()

In [2]:
import pandas as pd
import numpy as np
import scikit_posthocs as sp
from scipy.stats import friedmanchisquare
import warnings
warnings.filterwarnings("ignore")

A banca de qualificação solicitou que os testes sejam pareados entre si.

In [3]:
# Médias classificadores Sklearn
sklearn_scores = pd.read_csv(r'./results/2-0_anomaly_detection_scores_reais.csv', index_col=0)

# Médias classificadores Autoencoder
autoencoder_scores = pd.read_csv(r'./results/2-1_anomaly_detection_scores_reais.csv', index_col=0)

# Agrupa todos os resultados
scores = pd.concat([sklearn_scores, autoencoder_scores])

mean_score_table = scores.groupby('CLASSIFICADOR').mean().sort_values(by=['F1'], ascending=False)
mean_score_table

,PRECISAO,REVOGACAO,F1,TREINAMENTO [s],TESTE [s]
CLASSIFICADOR,,,,,
Local Outlier Factor,0.870370,0.870370,0.870370,0.001306,0.001302
Floresta de Isolamento (benchmark Vargas),0.722222,0.722222,0.722222,0.144691,0.031334
Floresta de Isolamento,0.701389,0.701389,0.701389,0.285244,0.061137
AutoEncoder,0.590278,0.590278,0.590278,1.441114,0.045269
Envelope Eliptico MCD,0.585648,0.585648,0.585648,0.070265,0.001199
Dummy,0.500000,0.500000,0.500000,0.000111,0.000083
One Class SVM (benchmark Vargas),0.486111,0.486111,0.486111,0.001141,0.000944
One Class SVM,0.476852,0.476852,0.476852,0.001190,0.000916


In [4]:
clfs_names = list(mean_score_table.index)
f1s = [scores.loc[scores['CLASSIFICADOR']==cn, 'F1'].values for cn in clfs_names]
clfs_names

['Local Outlier Factor',
 'Floresta de Isolamento (benchmark Vargas)',
 'Floresta de Isolamento',
 'AutoEncoder',
 'Envelope Eliptico MCD',
 'Dummy',
 'One Class SVM (benchmark Vargas)',
 'One Class SVM']

In [5]:
# Teste Estatístico (Friedman)

stat, p_value = friedmanchisquare(*(f1s))
print(f'p_value: {p_value}')

p_value: 2.4323213649592804e-20


In [6]:
# Teste Estatístico (Wilcoxon)

p_values = sp.posthoc_wilcoxon(f1s, 
                                 val_col=None, 
                                 group_col=None, 
                                 zero_method='wilcox', 
                                 correction=False, 
                                 p_adjust='bonferroni', 
                                 sort=False)

p_values.columns = clfs_names
p_values.index = clfs_names

In [7]:
df = pd.DataFrame(p_values)

d = dict(selector="th",
    props=[('text-align', 'center')])

df.style.set_properties(**{'width':'10em', 'text-align':'center'})\
        .set_table_styles([d])\
        .applymap(lambda x: 'color: green' if  x != '-' and x <= 0.05 and x > -1 else 'color: gray')\
        .set_caption("Valores p dos testes estatísticos Wilcoxon com ajuste de Bonferroni").set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('text-align', 'center'),
                ('font-size', '16px')
            ]
        }])

,Local Outlier Factor,Floresta de Isolamento (benchmark Vargas),Floresta de Isolamento,AutoEncoder,Envelope Eliptico MCD,Dummy,One Class SVM (benchmark Vargas),One Class SVM
Local Outlier Factor,1.000000,0.006755,0.000306,0.000006,0.000020,0.000005,0.000018,0.000024
Floresta de Isolamento (benchmark Vargas),0.006755,1.000000,1.000000,0.007872,0.010399,0.000127,0.006883,0.010787
Floresta de Isolamento,0.000306,1.000000,1.000000,0.065352,0.056257,0.000218,0.010718,0.007326
AutoEncoder,0.000006,0.007872,0.065352,1.000000,1.000000,0.112747,1.000000,1.000000
Envelope Eliptico MCD,0.000020,0.010399,0.056257,1.000000,1.000000,0.039093,1.000000,1.000000
Dummy,0.000005,0.000127,0.000218,0.112747,0.039093,1.000000,1.000000,1.000000
One Class SVM (benchmark Vargas),0.000018,0.006883,0.010718,1.000000,1.000000,1.000000,1.000000,1.000000
One Class SVM,0.000024,0.010787,0.007326,1.000000,1.000000,1.000000,1.000000,1.000000


## Conclusões

Tendo em vista os resultados apresentados acima, podemos afirmar que o classificador "LOF" apresenta o melhor desempenho em termos de médias da métrica medida-F1.

 - O classificador "LOF" produz um valor médio de medida-F1 estatisticamente diferente em comparação a todos os demais classificadores.

In [8]:
# Calcular tempo total do notebook Jupyter
print(f'Tempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')

Tempo total de execução (hh:mm:ss.ms): 0:00:01.846312
